In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column+1):
    if ws.cell(1, column).value == 'Src':
        src_index = column
    elif ws.cell(1, column).value == 'Pic':
        pic_index = column

print('Src索引：' + str(src_index))
print('Pic索引：' + str(pic_index))
print()     
print('总数量：' + str(ws.max_row-1))
print()     

# = = = = = = = = = = = = = = =

import requests
from io import BytesIO
from openpyxl.drawing.image import Image

from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(2, ws.max_row+1):
    work.put_nowait(i)

def crawler():
    global ws
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                src = ws.cell(i, src_index).value
                
                if src == '':
                    status = 'ok'
                    
                    break
                    
                pic = Image(BytesIO(requests.get(src,
                                                 proxies=Proxy.get_Proxy_Requests()).content))
                
                # = = = = = = = = = = = = = = =
                
                if pic.width >= pic.height:
                    pic.height = 50.0 / pic.width * pic.height
                    pic.width = 50.0
                else:
                    pic.width = 50.0 / pic.height * pic.width
                    pic.height = 50.0

                # = = = = = = = = = = = = = = =
                
                ws.add_image(pic, get_column_letter(pic_index) + str(i))

                status = 'ok'
                
                break
   
            except:
                time.sleep(0.5)
                continue

        print(src + '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
wb.save('./crawler-pic.xlsx')
print()
print('爬虫结束')

Src索引：11
Pic索引：8

总数量：58

https://www.rockauto.com/info/154/ADR114_Front.jpg  <->  [ok] - 剩余数量：38
https://www.rockauto.com/info/154/ADR4221_Front.jpg  <->  [ok] - 剩余数量：37
https://www.rockauto.com/info/154/ADR215_Front.jpg  <->  [ok] - 剩余数量：36
https://www.rockauto.com/info/154/ADR4280_Front.jpg  <->  [ok] - 剩余数量：35
https://www.rockauto.com/info/154/ADR4203_Front.jpg  <->  [ok] - 剩余数量：34
https://www.rockauto.com/info/154/ADR4291_Front.jpg  <->  [ok] - 剩余数量：33
https://www.rockauto.com/info/154/ADR4270_Front.jpg  <->  [ok] - 剩余数量：32
https://www.rockauto.com/info/154/ADR4207_Front.jpg  <->  [ok] - 剩余数量：31
https://www.rockauto.com/info/154/ADR4306_Front.jpg  <->  [ok] - 剩余数量：30
https://www.rockauto.com/info/154/ADR144_Front.jpg  <->  [ok] - 剩余数量：29
https://www.rockauto.com/info/154/ADR145_Front.jpg  <->  [ok] - 剩余数量：28
https://www.rockauto.com/info/154/ADR4277_Front.jpg  <->  [ok] - 剩余数量：27
https://www.rockauto.com/info/154/ADR4279_Front.jpg  <->  [ok] - 剩余数量：26
https://www.rockauto.com/info